## Exploring Battle Logs

In [1]:
from bs4 import BeautifulSoup



In [83]:
INPUT_DIR="../battle_data"
OUTPUT_DIR="../battle_data_parsed/v1"



In [128]:
class Player:
    def __init__(self, id, logs):
        self.id = id
        
        for log in logs:
            if log[0] == "player" and log[1] == id and len(log) > 2:
                self.player_name = log[2]
                break



In [129]:
UNKNOWN_POKEMON = 'UNKNOWN_POKEMON'

class Pokemon:
    def __init__(self, player, id, species, logs):
        self.id = id
        self.player = player
        self.species = species
        self._logs = logs


In [130]:
class PokemonTeam:
    def __init__(self, player, logs):
        self.player = player
        self._logs = logs
        self._pokemon = {}


    def __len__(self):
        for log in self._logs:
            if log[0] == 'teamsize' and log[1] == self.player.id:
                return int(log[2])

        return 0


    def _load_pokemon(self):
        prefix = self.player.id + "a: "
        count = len(self)
        found_count = 0

        for log in self._logs:
            player_prefix = self.player.id + "a: "
            if log[0] == 'switch' and log[1].startswith(player_prefix):
                p_id = log[1]

                if p_id in self._pokemon:
                    continue

                species = p_id.split(" ")[1]

                pokemon = Pokemon(player=self.player, id=p_id, species=species, logs=self._logs)

                self._pokemon[pokemon] = pokemon
                found_count += 1

        while found_count < count:
            p_id = prefix + UNKNOWN_POKEMON + " " + str(count - found_count)
            pokemon = Pokemon(player=self.player, id=p_id, species=UNKNOWN_POKEMON, logs=self._logs)
            self._pokemon[p_id] = pokemon
            found_count += 1




In [131]:
class BattleLogs:
    def __init__(self, logs):
        self._logs = logs
        self.p1 = Player(id="p1", logs=logs)
        self.p2 = Player(id="p2", logs=logs)

 
    @property
    def tier(self):
        for log in self._logs:
            if log[0] == "tier":
                return log[1]

    @property
    def is_rated(self):
        for log in log:
            if log[0] == 'rated':
                return True

        return False

    
    @property
    def rules(self):
        return [x[1] for x in self._logs if x[0] == 'rule']


    @property
    def p1_team(self):
        return PokemonTeam(player=self.p1, logs=self._logs)


    @property
    def p2_team(self):
        return PokemonTeam(player=self.p2, logs=self._logs)


    @property
    def winner(self):
        for log in self._logs:
            if log[0] == "win":
                player_id = log[1]
                break

        if self.p1.id == player_id:
            return self.p1
        elif self.p2.id == player_id:
            return self.p2
        else:
            return None



In [132]:
import pandas as pd

# Number of log files we have in our data.
LOGS_COUNT = 143

In [137]:
def generate_frame(logs_count):
    idx = ['p1', 'p2', 'team1_count', 'team2_count']
    df = pd.DataFrame()
    list = []

    for i in range(logs_count):
        filename = INPUT_DIR + "/" + str(i + 1).zfill(4) + ".html"

        with open(filename) as file:
                soup = BeautifulSoup(file, "html.parser")
                el = soup.find_all("script", class_="battle-log-data")[0]
                raw_logs = [x.split('|')[1:] for x in el.text.split('\n') if len(x) > 0]

        logs = BattleLogs(raw_logs)

        list.append(pd.Series([
            logs.p1.player_name,
            logs.p2.player_name,
            len(logs.p1_team),
            len(logs.p2_team),
        ]))

    df = pd.concat(list, axis=1).T
    df.columns = idx
    return df



In [140]:
df= generate_frame(70)
df

,p1,p2,team1_count,team2_count
0,Rhodes28,sbq1e,6,6
1,Xebrilag27,Rage69,6,6
2,Raccoonis,WhoIsJoeLigma?,6,6
3,bembi,SecondaryStomach,6,6
4,MrCrunchBar,Primainprime,6,6
...,...,...,...,...
65,Brentdv,AllLivesMatter88,6,6
66,Bryski3912,Masterctrl HHTA,6,6
67,Bluubus11,Naman125A,6,6
68,Omnimon merciful m,Fuminium,6,6
